<a href="https://colab.research.google.com/github/gomescheelsy03/Repo2/blob/main/Project6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import requests
import json

# Function to get the API token
def get_token(api_key, api_secret):
    auth_url = "https://api.petfinder.com/v2/oauth2/token"
    data = {
        "grant_type": "client_credentials",
        "client_id": api_key,
        "client_secret": api_secret
    }
    response = requests.post(auth_url, data=data)
    return response.json()['access_token']

# Function to make a general API request
def make_api_call(endpoint, token, params=None):
    headers = {
        "Authorization": f"Bearer {token}"
    }
    response = requests.get(f"https://api.petfinder.com/v2/{endpoint}", headers=headers, params=params)
    return response.json()

# Function to find animals in a specific zip code
def find_animals_by_zip(token, zip_code):
    return make_api_call("animals", token, params={"location": zip_code})

# Function to print animals data
def print_animals_data(data):
    animals = data.get('animals', [])
    if not animals:
        print("No animals found.")
        return
    for animal in animals:
        print(f"Name: {animal.get('name', 'No name available')}")
        # Safely extract breed information
        breed = animal.get('breeds', {}).get('primary', 'Breed information not available')
        print(f"Breed: {breed}")
        print(f"Age: {animal.get('age', 'Age information not available')}")
        print(f"Gender: {animal.get('gender', 'Gender information not available')}")
        print(f"URL: {animal.get('url', 'URL information not available')}")
        print("\n")  # Adds a new line for readability between different animals


# Function to print organization data
def print_organizations_data(data):
    for org in data.get('organizations', []):
        print(f"Name: {org['name']}")
        print(f"Email: {org.get('email', 'Not provided')}")
        city = org['address'].get('city', 'Not provided') if org['address'] else 'Not provided'
        state = org['address'].get('state', 'Not provided') if org['address'] else 'Not provided'
        postcode = org['address'].get('postcode', 'Not provided') if org['address'] else 'Not provided'
        print(f"City: {city}")
        print(f"State: {state}")
        print(f"Postal Code: {postcode}")
        print(f"Website: {org.get('website', 'Not provided')}")
        print(f"Mission Statement: {org.get('mission_statement', 'Not provided')}")
        print("\n")  # Adds a new line for readability between different organizations

# Main function to run the project
def main():
    api_key = "fNroQ326uA6Hu07BJJynNJ0mJSwVYCzmtGt36ywC9ySdPEHf8m"  # Replace with your API key
    api_secret = "tOmKKCxygroTgyU0L9nO4hEr69WofISRPozPA5aK"  # Replace with your API secret

    # Get API token
    token = get_token(api_key, api_secret)

    # Example of listing organizations in MD
    organizations_md = find_organizations_in_md(token)
    print_organizations_data(organizations_md)

    # User input for zip code
    zip_code = input("Enter a zip code to search for animals: ")
    animals_zip = find_animals_by_zip(token, zip_code)
    print_animals_data(animals_zip)

    # Add other API calls and print functions as needed

# Run the main function
if __name__ == "__main__":
    main()


Name: Little Zoo Sanctuary
Email: info@littlezoosanctuary.org
City: Crofton
State: MD
Postal Code: 21114
Website: https://www.littlezoosanctuary.org/
Mission Statement: Our mission is to rescue and provide safe harbor for all animals who are homeless, abused, or neglected so we...


Name: 1Love Sato Adoptions
Email: toby@tobysrescue.com
City: Chevy Chase
State: MD
Postal Code: 20815
Website: http://www.tobysrescue.com
Mission Statement: In February 2019, Moira was visiting Ana in Dorado, Puerto Rico when she showed me a photo of a pup...


Name: Tiny Toes Kitten Rescue, Inc.
Email: tinytoeskittenrescue@gmail.com
City: Great Mills
State: MD
Postal Code: 20634
Website: https://tinytoeskittenrescue.org
Mission Statement: None


Name: Pardoned Paws Rescue Inc
Email: info@pardonedpawsrescue.org
City: Earleville
State: MD
Postal Code: 21919
Website: https://www.pardonedpawsrescue.org
Mission Statement: PPR was started to save the less than perfect dogs.  The ones overlooked because they are 